In [2]:
import pandas as pd
import networkx as nx
import simpy
from tqdm.auto import tqdm

class Lane:
    def __init__(self, 
                 id_: str, 
                 num_blocks: int, 
                 each_block_length: float,
                 each_block_capacity: int):
        self.id_ = id_
        self.length = num_blocks * each_block_length
        self.each_block_capacity = each_block_capacity
        self.capacity = num_blocks * each_block_capacity
        self.num_blocks = num_blocks
        self.stats = {i: [] for i in range(num_blocks)}
        
    def __str__(self):
        return f"Lane {self.id_}: length={self.length}m, capacity={self.capacity}veh, blocks={self.num_blocks}"

    def __repr__(self):
        return self.__str__()

    def add_vehicle(self, block: int, id: str):
        self.stats[block].append(id)
    
    def remove_vehicle(self, block: int, id: str):
        self.stats[block].remove(id)
        
    def is_full(self, block: int) -> bool:
        return len(self.stats[block]) >= self.each_block_capacity
            
            
class Intersection:
    def __init__(self, directions: list):
        self.num_directions = len(directions)
        self.directions = directions
        self.light_stats = {direction: "red" for direction in directions}
        self.stats = -1
        self.update_lights_according_to_logic()
        
    def __str__(self):
        return f"Intersection: {self.light_stats}"
    
    def __repr__(self):
        return self.__str__()
        
    def change_light(self, direction: str, color: str):
        self.light_stats[direction] = color

    def update_lights_according_to_logic(self):
        self.stats = (self.stats + 1) % self.num_directions
        for i, direction in enumerate(self.directions):
            if i == self.stats:
                self.change_light(direction, "green")
            else:
                self.change_light(direction, "red")
    
class Stats:
    def __init__(self):
        self.data = pd.DataFrame(columns=["player_id", "arrived_time", "departed_time", "wait_time"])
        self.detailed_data = pd.DataFrame(columns=["time", "player_id", "path_from", "path_to", "block"])
    
    def record_trip(self, player_id: str, arrived_time: int, departed_time: int, wait_time: int):
        new_record = {
            "player_id": player_id,
            "arrived_time": arrived_time,
            "departed_time": departed_time,
            "wait_time": wait_time,
        }
        self.data = pd.concat([self.data, pd.DataFrame([new_record])], ignore_index=True)
        
    def record_detailed(self, time: int, player_id: str, path_from: str, path_to: str, block: str):
        new_record = {
            "time": time,
            "player_id": player_id,
            "path_from": path_from,
            "path_to": path_to,
            "block": block
        }
        self.detailed_data = pd.concat([self.detailed_data, pd.DataFrame([new_record])], ignore_index=True)
        
class Player:
    def __init__(self, id_: str, source: str, destination: str, graph: nx.DiGraph, stats: Stats, arrival_time: int, env: simpy.Environment, logged: bool = False):
        self.id_ = id_
        self.source = source
        self.destination = destination
        self.wait_time = 0
        self.graph = graph
        self.stats = stats
        self.logged = logged
        self.arrival_time = arrival_time
        self.departed = 0
        self.done = False
        self.env = env
        self.shortest_path = self._shortest_path_seeker()
        
        self.current_path = [self.source, self._get_next_node(self.source)]
        self.intersection = self.graph.nodes[self.current_path[1]]["intersection"]
        self.lane = self.graph.edges[(self.current_path[0], self.current_path[1])]["lane"]
        self.lane.add_vehicle(block=self.lane.num_blocks - 1, id=self.id_)
        self.block_position = self.lane.num_blocks - 1
        
    def _shortest_path_seeker(self):
        return nx.shortest_path(self.graph, source=self.source, target=self.destination, weight="travel_time")
        
    def _get_next_node(self, current_node: str) -> str:
        current_index = self.shortest_path.index(current_node)
        if current_index + 1 < len(self.shortest_path):
            return self.shortest_path[current_index + 1]
        return None

    def _move_forward(self):
        if self.lane.is_full(block=self.block_position-1):
            self.wait_time += 1
        else:
            self.lane.remove_vehicle(block=self.block_position, id=self.id_)
            self.block_position -= 1
            self.lane.add_vehicle(block=self.block_position, id=self.id_)
        self.log()
        
    def _move_intersection(self):
        if self.intersection.light_stats[self.current_path[0]] == "green":
            self.lane.remove_vehicle(block=self.block_position, id=self.id_)
            self.current_path[0] = self.current_path[1]
            self.current_path[1] = self._get_next_node(self.current_path[0])
            if self.current_path[1] is not None:
                self.intersection = self.graph.nodes[self.current_path[1]]["intersection"]
                self.lane = self.graph.edges[(self.current_path[0], self.current_path[1])]["lane"]
                self.block_position = self.lane.num_blocks - 1
                self.lane.add_vehicle(block=self.block_position, id=self.id_)
            else:
                self.departed = self.env.now
                self.done = True
                self.stats.record_trip(
                    player_id=self.id_,
                    arrived_time=self.arrival_time,
                    departed_time=self.departed,
                    wait_time=self.wait_time
                )
        else:
            self.wait_time += 1
        self.log()
        
    def make_decision(self):
        if self.done:
            return
        if self.block_position > 0:
            self._move_forward()
        elif self.block_position == 0:
            self._move_intersection()
        
    def log(self):
        if self.logged:
            self.stats.record_detailed(
                time=self.env.now,
                player_id=self.id_,
                path_from=self.current_path[0],
                path_to=self.current_path[1],
                block=self.block_position
            )

## LoadData Class Implementation

In [3]:
class LoadData:
    def __init__(self):
        self.graph = None
        self.demand = None
    
    def load_network(self, network_file: str, position_file: str, each_block_length=100, each_block_capacity=20):
        data_xy = pd.read_csv(network_file)
        data_xy_pos = pd.read_csv(position_file, index_col=0)
        data_xy_pos = data_xy_pos.set_index('node').apply(lambda row: (row.x, row.y), axis=1).to_dict()
        speed = 80  # km/h
        data_xy_traveltime = {(int(row["from"]), int(row["to"])): row["length"]/(speed*1000/60) for _, row in data_xy.iterrows()} # minutes
        
        graph = nx.from_pandas_edgelist(data_xy, source="from", target="to", edge_attr=True, create_using=nx.DiGraph())
        nx.set_node_attributes(graph, data_xy_pos, 'pos')
        nx.set_edge_attributes(graph, data_xy_traveltime, 'travel_time')
        
        for edge in graph.edges(data=True):
            lane_id = 0
            num_blocks = edge[2]["length"] // 100
            lane = Lane(id_=lane_id, num_blocks=num_blocks, 
                        each_block_length=each_block_length, 
                        each_block_capacity=each_block_capacity)
            graph.edges[(edge[0], edge[1])]["lane"] = lane
            
        for node in graph.nodes:
            neighbors = list(graph.successors(node))
            intersection = Intersection(directions=neighbors)
            graph.nodes[node]["intersection"] = intersection
            
        self.graph = graph

    def load_demand(self, demand_file: str):
        self.demand = pd.read_csv(demand_file)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime

class Analysis:
    def __init__(self, data: pd.DataFrame, detailed_data: pd.DataFrame):
        self.detailed_data = detailed_data
        self.data = data
        
    def plot_trajectory(self, lane_id: tuple[str, str], save_path: str = None) -> None:
        """
        Plot time-space diagram (trajectory) for vehicles on a specific lane.
        lane_id: tuple of (path_from, path_to) representing the edge/lane
        """
        df = self.detailed_data
        
        # Filter data for the specific lane
        lane_data = df[(df["path_from"] == lane_id[0]) & (df["path_to"] == lane_id[1])]
        
        if lane_data.empty:
            print(f"No data found for lane {lane_id}")
            return
        
        plt.figure(figsize=(12, 6))
        
        # Plot trajectory for each player on this lane
        for player_id in lane_data["player_id"].unique():
            player_data = lane_data[lane_data["player_id"] == player_id].sort_values("time")
            plt.plot(player_data["time"], player_data["block"], 
                    marker='o', markersize=2, alpha=0.7, linewidth=1)
        
        plt.xlabel("Time (s)")
        plt.ylabel("Block Position")
        plt.title(f"Time-Space Diagram for Lane ({lane_id[0]} → {lane_id[1]})")
        plt.gca().invert_yaxis()  # Block 0 is at intersection, so invert for intuitive view
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def plot_wait_time_distribution(self, save_path: str = None) -> None:
        """
        Plot the distribution of wait times for all completed trips.
        """
        df = self.data
        
        if df.empty:
            print("No trip data available")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Histogram of wait times
        axes[0].hist(df["wait_time"], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        axes[0].set_xlabel("Wait Time (time units)")
        axes[0].set_ylabel("Frequency")
        axes[0].set_title("Distribution of Wait Times")
        axes[0].axvline(df["wait_time"].mean(), color='red', linestyle='--', 
                       label=f'Mean: {df["wait_time"].mean():.2f}')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Box plot
        axes[1].boxplot(df["wait_time"], vert=True)
        axes[1].set_ylabel("Wait Time (time units)")
        axes[1].set_title("Wait Time Box Plot")
        axes[1].grid(True, alpha=0.3)
        
        # Add statistics text
        stats_text = f"Mean: {df['wait_time'].mean():.2f}\n"
        stats_text += f"Median: {df['wait_time'].median():.2f}\n"
        stats_text += f"Std: {df['wait_time'].std():.2f}\n"
        stats_text += f"Max: {df['wait_time'].max():.2f}"
        axes[1].text(1.15, df['wait_time'].median(), stats_text, fontsize=10,
                    verticalalignment='center')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
        
        # Also show travel time distribution
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        plt.figure(figsize=(10, 5))
        plt.hist(df["travel_time"], bins=30, edgecolor='black', alpha=0.7, color='green')
        plt.xlabel("Travel Time (time units)")
        plt.ylabel("Frequency")
        plt.title("Distribution of Total Travel Times")
        plt.axvline(df["travel_time"].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df["travel_time"].mean():.2f}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        if save_path:
            travel_save_path = save_path.replace('.png', '_travel_time.png')
            plt.savefig(travel_save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def plot_queue_length_over_time(self, lane_id: tuple[str, str], save_path: str = None) -> dict:
        """
        Plot cumulative arrival-departure diagram (input-output diagram) for a specific lane.
        Returns metrics dictionary for the lane.
        """
        df = self.detailed_data.copy()
        
        # Filter data for the specific lane
        lane_data = df[(df["path_from"] == lane_id[0]) & (df["path_to"] == lane_id[1])]
        
        if lane_data.empty:
            return None
        
        # Get unique players on this lane
        players_on_lane = lane_data["player_id"].unique()
        
        arrivals = []
        departures = []
        
        for player_id in players_on_lane:
            player_data = lane_data[lane_data["player_id"] == player_id].sort_values("time")
            block_0_data = player_data[player_data["block"] == 0]
            if not block_0_data.empty:
                arrival_time = block_0_data["time"].min()
                departure_time = block_0_data["time"].max()
                arrivals.append(arrival_time)
                departures.append(departure_time)
        
        if not arrivals:
            return None
        
        arrivals = sorted(arrivals)
        departures = sorted(departures)
        
        time_min = min(min(arrivals), min(departures))
        time_max = max(max(arrivals), max(departures))
        time_range = np.arange(time_min, time_max + 1)
        
        cum_arrivals = np.array([sum(1 for a in arrivals if a <= t) for t in time_range])
        cum_departures = np.array([sum(1 for d in departures if d <= t) for t in time_range])
        
        queue_length = cum_arrivals - cum_departures
        total_delay = np.trapz(queue_length, time_range)
        avg_delay_per_vehicle = total_delay / len(arrivals) if arrivals else 0
        max_queue = queue_length.max()
        
        # Store metrics
        metrics = {
            "lane_from": lane_id[0],
            "lane_to": lane_id[1],
            "total_vehicles": len(arrivals),
            "total_delay": total_delay,
            "avg_delay_per_vehicle": avg_delay_per_vehicle,
            "max_queue_length": max_queue,
            "mean_queue_length": queue_length.mean()
        }
        
        # Plot
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        axes[0].step(time_range, cum_arrivals, where='post', label='Cumulative Arrivals (A(t))', 
                    color='blue', linewidth=2)
        axes[0].step(time_range, cum_departures, where='post', label='Cumulative Departures (D(t))', 
                    color='red', linewidth=2)
        axes[0].fill_between(time_range, cum_arrivals, cum_departures, 
                            alpha=0.3, color='orange', label='Queue (Delay Area)')
        
        axes[0].set_xlabel("Time (s)")
        axes[0].set_ylabel("Cumulative Vehicle Count")
        axes[0].set_title(f"Cumulative Arrival-Departure Diagram for Lane ({lane_id[0]} → {lane_id[1]})")
        axes[0].legend(loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        stats_text = f"Total Delay: {total_delay:.0f} veh·s\n"
        stats_text += f"Avg Delay/Vehicle: {avg_delay_per_vehicle:.1f} s\n"
        stats_text += f"Max Queue Length: {max_queue} veh\n"
        stats_text += f"Total Vehicles: {len(arrivals)}"
        axes[0].text(0.98, 0.02, stats_text, transform=axes[0].transAxes, fontsize=10,
                    verticalalignment='bottom', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        axes[1].fill_between(time_range, queue_length, alpha=0.5, color='green', label='Queue Length')
        axes[1].plot(time_range, queue_length, color='darkgreen', linewidth=1.5)
        axes[1].axhline(y=queue_length.mean(), color='red', linestyle='--', 
                       label=f'Mean Queue: {queue_length.mean():.1f}')
        
        axes[1].set_xlabel("Time (s)")
        axes[1].set_ylabel("Queue Length (vehicles)")
        axes[1].set_title(f"Queue Length Over Time for Lane ({lane_id[0]} → {lane_id[1]})")
        axes[1].legend(loc='upper right')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
        
        return metrics

    def run_full_network_analysis(self, output_dir: str = "analysis_results") -> dict:
        """
        Run comprehensive analysis on all lanes in the network.
        Generates trajectory plots, queue plots, and summary statistics for all lanes.
        Saves all plots and data to the specified output directory.
        
        Parameters:
        -----------
        output_dir : str
            Directory path where all outputs will be saved
            
        Returns:
        --------
        dict : Dictionary containing all summary statistics and dataframes
        """
        # Create output directories
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = os.path.join(output_dir, f"analysis_{timestamp}")
        trajectory_dir = os.path.join(base_dir, "trajectories")
        queue_dir = os.path.join(base_dir, "queue_analysis")
        summary_dir = os.path.join(base_dir, "summary")
        
        for dir_path in [trajectory_dir, queue_dir, summary_dir]:
            os.makedirs(dir_path, exist_ok=True)
        
        print(f"📁 Output directory: {base_dir}")
        print("=" * 60)
        
        # Get all unique lanes from detailed data
        lanes = self.detailed_data.groupby(["path_from", "path_to"]).size().reset_index(name='count')
        lanes = lanes.sort_values('count', ascending=False)
        total_lanes = len(lanes)
        
        print(f"🔍 Found {total_lanes} lanes with vehicle data")
        print("=" * 60)
        
        # Store all lane metrics
        lane_metrics_list = []
        
        # Process each lane
        for idx, row in tqdm(lanes.iterrows(), total=total_lanes, desc="Processing lanes"):
            lane_id = (row["path_from"], row["path_to"])
            lane_str = f"{lane_id[0]}_to_{lane_id[1]}"
            
            # Generate trajectory plot
            traj_path = os.path.join(trajectory_dir, f"trajectory_{lane_str}.png")
            self.plot_trajectory(lane_id=lane_id, save_path=traj_path)
            
            # Generate queue analysis plot and get metrics
            queue_path = os.path.join(queue_dir, f"queue_{lane_str}.png")
            metrics = self.plot_queue_length_over_time(lane_id=lane_id, save_path=queue_path)
            
            if metrics:
                lane_metrics_list.append(metrics)
        
        # Create lane metrics dataframe
        lane_metrics_df = pd.DataFrame(lane_metrics_list)
        
        # Calculate network-wide statistics
        network_stats = self._calculate_network_statistics()
        
        # Generate summary plots
        self._generate_summary_plots(lane_metrics_df, network_stats, summary_dir)
        
        # Save all dataframes
        self._save_dataframes(lane_metrics_df, network_stats, summary_dir)
        
        # Generate final report
        self._generate_report(lane_metrics_df, network_stats, base_dir)
        
        print("\n" + "=" * 60)
        print("✅ Analysis Complete!")
        print(f"📊 All results saved to: {base_dir}")
        print("=" * 60)
        
        return {
            "lane_metrics": lane_metrics_df,
            "network_stats": network_stats,
            "output_dir": base_dir
        }
    
    def _calculate_network_statistics(self) -> dict:
        """Calculate network-wide statistics from trip data."""
        df = self.data.copy()
        
        if df.empty:
            return {}
        
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        stats = {
            # Trip statistics
            "total_trips_completed": len(df),
            "avg_wait_time": df["wait_time"].mean(),
            "median_wait_time": df["wait_time"].median(),
            "std_wait_time": df["wait_time"].std(),
            "min_wait_time": df["wait_time"].min(),
            "max_wait_time": df["wait_time"].max(),
            "percentile_25_wait": df["wait_time"].quantile(0.25),
            "percentile_75_wait": df["wait_time"].quantile(0.75),
            "percentile_95_wait": df["wait_time"].quantile(0.95),
            
            # Travel time statistics
            "avg_travel_time": df["travel_time"].mean(),
            "median_travel_time": df["travel_time"].median(),
            "std_travel_time": df["travel_time"].std(),
            "min_travel_time": df["travel_time"].min(),
            "max_travel_time": df["travel_time"].max(),
            "percentile_25_travel": df["travel_time"].quantile(0.25),
            "percentile_75_travel": df["travel_time"].quantile(0.75),
            "percentile_95_travel": df["travel_time"].quantile(0.95),
            
            # Throughput
            "simulation_duration": df["departed_time"].max() - df["arrived_time"].min(),
            "throughput_per_hour": len(df) / ((df["departed_time"].max() - df["arrived_time"].min()) / 3600) if df["departed_time"].max() > df["arrived_time"].min() else 0,
        }
        
        return stats
    
    def _generate_summary_plots(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate summary plots for the entire network."""
        
        # 1. Wait time and Travel time distribution (combined)
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        # Wait time histogram
        axes[0, 0].hist(df["wait_time"], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
        axes[0, 0].axvline(df["wait_time"].mean(), color='red', linestyle='--', linewidth=2,
                          label=f'Mean: {df["wait_time"].mean():.1f}')
        axes[0, 0].axvline(df["wait_time"].median(), color='orange', linestyle='-.', linewidth=2,
                          label=f'Median: {df["wait_time"].median():.1f}')
        axes[0, 0].set_xlabel("Wait Time (time units)", fontsize=12)
        axes[0, 0].set_ylabel("Frequency", fontsize=12)
        axes[0, 0].set_title("Network-Wide Wait Time Distribution", fontsize=14, fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Travel time histogram
        axes[0, 1].hist(df["travel_time"], bins=50, edgecolor='black', alpha=0.7, color='forestgreen')
        axes[0, 1].axvline(df["travel_time"].mean(), color='red', linestyle='--', linewidth=2,
                          label=f'Mean: {df["travel_time"].mean():.1f}')
        axes[0, 1].axvline(df["travel_time"].median(), color='orange', linestyle='-.', linewidth=2,
                          label=f'Median: {df["travel_time"].median():.1f}')
        axes[0, 1].set_xlabel("Travel Time (time units)", fontsize=12)
        axes[0, 1].set_ylabel("Frequency", fontsize=12)
        axes[0, 1].set_title("Network-Wide Travel Time Distribution", fontsize=14, fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Combined box plots
        box_data = [df["wait_time"].dropna(), df["travel_time"].dropna()]
        bp = axes[1, 0].boxplot(box_data, labels=['Wait Time', 'Travel Time'], patch_artist=True)
        colors = ['steelblue', 'forestgreen']
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        axes[1, 0].set_ylabel("Time (time units)", fontsize=12)
        axes[1, 0].set_title("Wait Time vs Travel Time Comparison", fontsize=14, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # Arrival time vs travel time scatter
        axes[1, 1].scatter(df["arrived_time"], df["travel_time"], alpha=0.3, s=10, c='purple')
        axes[1, 1].set_xlabel("Arrival Time (time units)", fontsize=12)
        axes[1, 1].set_ylabel("Travel Time (time units)", fontsize=12)
        axes[1, 1].set_title("Travel Time vs Arrival Time", fontsize=14, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "network_time_distributions.png"), dpi=200, bbox_inches='tight')
        plt.close()
        
        # 2. Lane-level analysis plots
        if not lane_metrics_df.empty:
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            
            # Top 20 lanes by total delay
            top_delay = lane_metrics_df.nlargest(20, 'total_delay')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_delay.iterrows()]
            axes[0, 0].barh(range(len(top_delay)), top_delay['total_delay'], color='coral', edgecolor='black')
            axes[0, 0].set_yticks(range(len(top_delay)))
            axes[0, 0].set_yticklabels(lane_labels, fontsize=8)
            axes[0, 0].set_xlabel("Total Delay (veh·s)", fontsize=12)
            axes[0, 0].set_title("Top 20 Lanes by Total Delay", fontsize=14, fontweight='bold')
            axes[0, 0].grid(True, alpha=0.3, axis='x')
            axes[0, 0].invert_yaxis()
            
            # Top 20 lanes by max queue
            top_queue = lane_metrics_df.nlargest(20, 'max_queue_length')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_queue.iterrows()]
            axes[0, 1].barh(range(len(top_queue)), top_queue['max_queue_length'], color='teal', edgecolor='black')
            axes[0, 1].set_yticks(range(len(top_queue)))
            axes[0, 1].set_yticklabels(lane_labels, fontsize=8)
            axes[0, 1].set_xlabel("Max Queue Length (vehicles)", fontsize=12)
            axes[0, 1].set_title("Top 20 Lanes by Max Queue Length", fontsize=14, fontweight='bold')
            axes[0, 1].grid(True, alpha=0.3, axis='x')
            axes[0, 1].invert_yaxis()
            
            # Vehicle count distribution across lanes
            axes[1, 0].hist(lane_metrics_df['total_vehicles'], bins=30, edgecolor='black', alpha=0.7, color='goldenrod')
            axes[1, 0].axvline(lane_metrics_df['total_vehicles'].mean(), color='red', linestyle='--', linewidth=2,
                              label=f'Mean: {lane_metrics_df["total_vehicles"].mean():.1f}')
            axes[1, 0].set_xlabel("Number of Vehicles", fontsize=12)
            axes[1, 0].set_ylabel("Number of Lanes", fontsize=12)
            axes[1, 0].set_title("Distribution of Vehicle Counts per Lane", fontsize=14, fontweight='bold')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
            
            # Scatter: vehicles vs avg delay
            axes[1, 1].scatter(lane_metrics_df['total_vehicles'], lane_metrics_df['avg_delay_per_vehicle'], 
                              alpha=0.6, s=50, c='darkviolet', edgecolors='black', linewidth=0.5)
            axes[1, 1].set_xlabel("Total Vehicles", fontsize=12)
            axes[1, 1].set_ylabel("Average Delay per Vehicle (s)", fontsize=12)
            axes[1, 1].set_title("Vehicle Volume vs Average Delay per Vehicle", fontsize=14, fontweight='bold')
            axes[1, 1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, "lane_level_analysis.png"), dpi=200, bbox_inches='tight')
            plt.close()
        
        # 3. Time series analysis
        fig, axes = plt.subplots(2, 1, figsize=(16, 10))
        
        # Arrivals over time
        arrival_counts = df.groupby('arrived_time').size()
        window = 60  # 1 minute rolling window
        arrival_rate = arrival_counts.rolling(window=window, min_periods=1).sum()
        
        axes[0].plot(arrival_rate.index, arrival_rate.values, color='blue', linewidth=1.5, alpha=0.8)
        axes[0].fill_between(arrival_rate.index, arrival_rate.values, alpha=0.3, color='blue')
        axes[0].set_xlabel("Time (s)", fontsize=12)
        axes[0].set_ylabel(f"Arrivals (per {window}s window)", fontsize=12)
        axes[0].set_title("Vehicle Arrival Rate Over Time", fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        
        # Departures over time
        departure_counts = df.groupby('departed_time').size()
        departure_rate = departure_counts.rolling(window=window, min_periods=1).sum()
        
        axes[1].plot(departure_rate.index, departure_rate.values, color='green', linewidth=1.5, alpha=0.8)
        axes[1].fill_between(departure_rate.index, departure_rate.values, alpha=0.3, color='green')
        axes[1].set_xlabel("Time (s)", fontsize=12)
        axes[1].set_ylabel(f"Departures (per {window}s window)", fontsize=12)
        axes[1].set_title("Vehicle Departure Rate Over Time", fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "temporal_analysis.png"), dpi=200, bbox_inches='tight')
        plt.close()
        
        # 4. Comprehensive dashboard
        self._generate_dashboard(lane_metrics_df, network_stats, output_dir)
    
    def _generate_dashboard(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate a comprehensive dashboard summary."""
        fig = plt.figure(figsize=(20, 16))
        
        # Create grid for dashboard
        gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
        
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        
        # Title
        fig.suptitle("🚗 Traffic Simulation Analysis Dashboard", fontsize=20, fontweight='bold', y=0.98)
        
        # 1. Key metrics boxes (top row)
        ax_metrics = fig.add_subplot(gs[0, :])
        ax_metrics.axis('off')
        
        metrics_text = (
            f"📊 NETWORK SUMMARY STATISTICS\n\n"
            f"Total Trips Completed: {network_stats.get('total_trips_completed', 0):,}\n"
            f"Average Wait Time: {network_stats.get('avg_wait_time', 0):.2f} s\n"
            f"Average Travel Time: {network_stats.get('avg_travel_time', 0):.2f} s\n"
            f"95th Percentile Wait Time: {network_stats.get('percentile_95_wait', 0):.2f} s\n"
            f"Throughput: {network_stats.get('throughput_per_hour', 0):.0f} veh/hour\n"
            f"Total Lanes Analyzed: {len(lane_metrics_df)}"
        )
        
        ax_metrics.text(0.5, 0.5, metrics_text, transform=ax_metrics.transAxes, fontsize=14,
                       verticalalignment='center', horizontalalignment='center',
                       bbox=dict(boxstyle='round,pad=1', facecolor='lightblue', alpha=0.8),
                       family='monospace')
        
        # 2. Wait time distribution
        ax1 = fig.add_subplot(gs[1, 0:2])
        ax1.hist(df["wait_time"], bins=40, edgecolor='black', alpha=0.7, color='steelblue')
        ax1.axvline(df["wait_time"].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax1.set_xlabel("Wait Time (s)")
        ax1.set_ylabel("Frequency")
        ax1.set_title("Wait Time Distribution", fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 3. Travel time distribution
        ax2 = fig.add_subplot(gs[1, 2:4])
        ax2.hist(df["travel_time"], bins=40, edgecolor='black', alpha=0.7, color='forestgreen')
        ax2.axvline(df["travel_time"].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax2.set_xlabel("Travel Time (s)")
        ax2.set_ylabel("Frequency")
        ax2.set_title("Travel Time Distribution", fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # 4. Top 10 congested lanes
        ax3 = fig.add_subplot(gs[2, 0:2])
        if not lane_metrics_df.empty:
            top_delay = lane_metrics_df.nlargest(10, 'total_delay')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_delay.iterrows()]
            bars = ax3.barh(range(len(top_delay)), top_delay['total_delay'], color='coral', edgecolor='black')
            ax3.set_yticks(range(len(top_delay)))
            ax3.set_yticklabels(lane_labels, fontsize=9)
            ax3.set_xlabel("Total Delay (veh·s)")
            ax3.set_title("Top 10 Most Congested Lanes", fontweight='bold')
            ax3.invert_yaxis()
            ax3.grid(True, alpha=0.3, axis='x')
        
        # 5. Top 10 lanes by queue
        ax4 = fig.add_subplot(gs[2, 2:4])
        if not lane_metrics_df.empty:
            top_queue = lane_metrics_df.nlargest(10, 'max_queue_length')
            lane_labels = [f"{r['lane_from']}→{r['lane_to']}" for _, r in top_queue.iterrows()]
            ax4.barh(range(len(top_queue)), top_queue['max_queue_length'], color='teal', edgecolor='black')
            ax4.set_yticks(range(len(top_queue)))
            ax4.set_yticklabels(lane_labels, fontsize=9)
            ax4.set_xlabel("Max Queue Length (vehicles)")
            ax4.set_title("Top 10 Lanes by Max Queue", fontweight='bold')
            ax4.invert_yaxis()
            ax4.grid(True, alpha=0.3, axis='x')
        
        # 6. Arrival/Departure rates over time
        ax5 = fig.add_subplot(gs[3, 0:2])
        window = 120
        arrival_counts = df.groupby('arrived_time').size()
        departure_counts = df.groupby('departed_time').size()
        
        all_times = pd.Index(range(int(df['arrived_time'].min()), int(df['departed_time'].max()) + 1))
        arrival_rate = arrival_counts.reindex(all_times, fill_value=0).rolling(window=window, min_periods=1).sum()
        departure_rate = departure_counts.reindex(all_times, fill_value=0).rolling(window=window, min_periods=1).sum()
        
        ax5.plot(arrival_rate.index, arrival_rate.values, color='blue', linewidth=1.5, label='Arrivals', alpha=0.8)
        ax5.plot(departure_rate.index, departure_rate.values, color='green', linewidth=1.5, label='Departures', alpha=0.8)
        ax5.fill_between(arrival_rate.index, arrival_rate.values, departure_rate.values, 
                        where=arrival_rate.values > departure_rate.values, alpha=0.3, color='red', label='Queue Building')
        ax5.set_xlabel("Time (s)")
        ax5.set_ylabel(f"Vehicles (per {window}s)")
        ax5.set_title("Arrival vs Departure Rate", fontweight='bold')
        ax5.legend(loc='upper right', fontsize=8)
        ax5.grid(True, alpha=0.3)
        
        # 7. Cumulative trips
        ax6 = fig.add_subplot(gs[3, 2:4])
        cum_arrivals = arrival_counts.reindex(all_times, fill_value=0).cumsum()
        cum_departures = departure_counts.reindex(all_times, fill_value=0).cumsum()
        
        ax6.plot(cum_arrivals.index, cum_arrivals.values, color='blue', linewidth=2, label='Cumulative Arrivals')
        ax6.plot(cum_departures.index, cum_departures.values, color='green', linewidth=2, label='Cumulative Departures')
        ax6.fill_between(cum_arrivals.index, cum_arrivals.values, cum_departures.values, alpha=0.3, color='orange')
        ax6.set_xlabel("Time (s)")
        ax6.set_ylabel("Cumulative Vehicles")
        ax6.set_title("Cumulative Arrivals vs Departures", fontweight='bold')
        ax6.legend(loc='upper left')
        ax6.grid(True, alpha=0.3)
        
        plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')
        plt.close()
    
    def _save_dataframes(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Save all analysis dataframes to CSV."""
        
        # 1. Lane metrics
        lane_metrics_df.to_csv(os.path.join(output_dir, "lane_metrics.csv"), index=False)
        
        # 2. Network statistics
        stats_df = pd.DataFrame([network_stats])
        stats_df.to_csv(os.path.join(output_dir, "network_statistics.csv"), index=False)
        
        # 3. Trip-level data with travel time
        df = self.data.copy()
        df["travel_time"] = df["departed_time"] - df["arrived_time"]
        df.to_csv(os.path.join(output_dir, "trip_data_with_travel_time.csv"), index=False)
        
        # 4. Detailed movement data
        self.detailed_data.to_csv(os.path.join(output_dir, "detailed_movement_data.csv"), index=False)
        
        # 5. Summary by lane
        lane_summary = self.detailed_data.groupby(['path_from', 'path_to']).agg({
            'player_id': 'nunique',
            'time': ['min', 'max'],
            'block': ['min', 'max', 'mean']
        }).reset_index()
        lane_summary.columns = ['lane_from', 'lane_to', 'unique_vehicles', 
                               'first_observation', 'last_observation',
                               'min_block', 'max_block', 'avg_block']
        lane_summary.to_csv(os.path.join(output_dir, "lane_summary.csv"), index=False)
        
        print(f"📄 Saved CSV files to {output_dir}")
    
    def _generate_report(self, lane_metrics_df: pd.DataFrame, network_stats: dict, output_dir: str):
        """Generate a text report summarizing the analysis."""
        
        report_lines = [
            "=" * 80,
            "TRAFFIC SIMULATION ANALYSIS REPORT",
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            "=" * 80,
            "",
            "NETWORK-WIDE STATISTICS",
            "-" * 40,
            f"Total Trips Completed: {network_stats.get('total_trips_completed', 0):,}",
            f"Simulation Duration: {network_stats.get('simulation_duration', 0):.0f} seconds",
            f"Throughput: {network_stats.get('throughput_per_hour', 0):.0f} vehicles/hour",
            "",
            "WAIT TIME STATISTICS",
            "-" * 40,
            f"Average Wait Time: {network_stats.get('avg_wait_time', 0):.2f} seconds",
            f"Median Wait Time: {network_stats.get('median_wait_time', 0):.2f} seconds",
            f"Std Dev Wait Time: {network_stats.get('std_wait_time', 0):.2f} seconds",
            f"Min Wait Time: {network_stats.get('min_wait_time', 0):.2f} seconds",
            f"Max Wait Time: {network_stats.get('max_wait_time', 0):.2f} seconds",
            f"25th Percentile: {network_stats.get('percentile_25_wait', 0):.2f} seconds",
            f"75th Percentile: {network_stats.get('percentile_75_wait', 0):.2f} seconds",
            f"95th Percentile: {network_stats.get('percentile_95_wait', 0):.2f} seconds",
            "",
            "TRAVEL TIME STATISTICS",
            "-" * 40,
            f"Average Travel Time: {network_stats.get('avg_travel_time', 0):.2f} seconds",
            f"Median Travel Time: {network_stats.get('median_travel_time', 0):.2f} seconds",
            f"Std Dev Travel Time: {network_stats.get('std_travel_time', 0):.2f} seconds",
            f"Min Travel Time: {network_stats.get('min_travel_time', 0):.2f} seconds",
            f"Max Travel Time: {network_stats.get('max_travel_time', 0):.2f} seconds",
            f"25th Percentile: {network_stats.get('percentile_25_travel', 0):.2f} seconds",
            f"75th Percentile: {network_stats.get('percentile_75_travel', 0):.2f} seconds",
            f"95th Percentile: {network_stats.get('percentile_95_travel', 0):.2f} seconds",
            "",
            "LANE-LEVEL STATISTICS",
            "-" * 40,
            f"Total Lanes Analyzed: {len(lane_metrics_df)}",
        ]
        
        if not lane_metrics_df.empty:
            report_lines.extend([
                f"Total Network Delay: {lane_metrics_df['total_delay'].sum():.0f} vehicle-seconds",
                f"Average Delay per Lane: {lane_metrics_df['total_delay'].mean():.2f} vehicle-seconds",
                f"Max Single Lane Delay: {lane_metrics_df['total_delay'].max():.0f} vehicle-seconds",
                f"Average Max Queue Length: {lane_metrics_df['max_queue_length'].mean():.2f} vehicles",
                f"Highest Max Queue: {lane_metrics_df['max_queue_length'].max():.0f} vehicles",
                "",
                "TOP 10 MOST CONGESTED LANES (by Total Delay)",
                "-" * 40,
            ])
            
            top_10 = lane_metrics_df.nlargest(10, 'total_delay')
            for i, (_, row) in enumerate(top_10.iterrows(), 1):
                report_lines.append(
                    f"{i:2}. Lane {row['lane_from']} → {row['lane_to']}: "
                    f"Delay={row['total_delay']:.0f}s, MaxQueue={row['max_queue_length']:.0f}veh, "
                    f"Vehicles={row['total_vehicles']:.0f}"
                )
        
        report_lines.extend([
            "",
            "=" * 80,
            "OUTPUT FILES",
            "-" * 40,
            "- lane_metrics.csv: Metrics for each lane",
            "- network_statistics.csv: Network-wide statistics",
            "- trip_data_with_travel_time.csv: All trip records",
            "- detailed_movement_data.csv: Vehicle movement logs",
            "- lane_summary.csv: Summary statistics per lane",
            "- comprehensive_dashboard.png: Visual dashboard",
            "- network_time_distributions.png: Time distribution plots",
            "- lane_level_analysis.png: Lane-level analysis plots",
            "- temporal_analysis.png: Time series analysis",
            "- trajectories/: Individual lane trajectory plots",
            "- queue_analysis/: Individual lane queue plots",
            "=" * 80,
        ])
        
        report_text = "\n".join(report_lines)
        
        with open(os.path.join(output_dir, "analysis_report.txt"), 'w') as f:
            f.write(report_text)
        
        print("\n" + report_text)

Matplotlib is building the font cache; this may take a moment.


## Clock Class Implementation

In [5]:
class Clock:
    def __init__(self, env: simpy.Environment, world: LoadData):
        self.env = env
        self.graph = world.graph
        self.demand = world.demand
        self.stats = Stats()
        self.player = {}

    def generate_player(self) -> simpy.events.Generator:
        with tqdm(total=self.duration, desc="Simulation Progress") as pbar:
            last_time = 0
            while True:
                current_time = self.env.now

                if current_time % 5 < 0.01:
                    for start, end, data in self.graph.edges(data=True):
                        for k, v in data["lane"].stats.items():
                            for v_id in list(v):
                                self.player[v_id].make_decision()
                        
                if current_time % 45 < 0.01:
                    for _, intersection in self.graph.nodes(data="intersection"):
                        intersection.update_lights_according_to_logic()

                arriving_players = self.demand[self.demand["arrival_time"] == current_time]
                for i, row in arriving_players.iterrows():
                    # if i % 3 == 0:
                    #     logged = True
                    # else:
                    #     logged = False
                    logged = True
                    player = Player(
                        env=self.env,
                        id_=row["player_id"],
                        source=row["source"],
                        destination=row["destination"],
                        graph=self.graph,
                        stats=self.stats,
                        arrival_time=row["arrival_time"],
                        logged=logged
                    )
                    self.player[player.id_] = player

                # Update tqdm progress bar
                pbar.update(current_time - last_time)
                last_time = current_time

                yield self.env.timeout(1)
    
    def run_simulation(self, duration: int) -> simpy.events.Generator:
        self.duration = duration
        self.env.process(self.generate_player())
        self.env.run(until=duration)
        


In [7]:
network_file = "data/network/SiouxFallsNetwork.csv"
position_file = "data/network/SiouxFallsXY.csv"
demand_file = "/Users/keivanjamali/Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/demand.csv"
# demand_file = "/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/test/demand2.csv"

# test scenario
# network_file = "data/network_test/SiouxFallsNetwork.csv"
# position_file = "data/network_test/SiouxFallsXY.csv"
# demand_file = "data/network_test/demand.csv"

world = LoadData()
world.load_network(network_file=network_file, position_file=position_file,
                   each_block_length=100, each_block_capacity=1)
world.load_demand(demand_file=demand_file)
clock = Clock(env=simpy.Environment(), world=world)
clock.run_simulation(duration=3600)

Simulation Progress: 100%|█████████▉| 3595/3600 [16:28<00:00, 11.15it/s]

In [23]:
data = clock.stats

data.detailed_data.to_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result/result_detailed3.csv", index=False)
data.data.to_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result/result_summary3.csv", index=False)
data.detailed_data

,time,player_id,path_from,path_to,block
0,5,1,1,2,28
1,5,3,1,2,29
2,5,4,1,2,29
3,5,2,2,1,28
4,5,5,2,1,29
...,...,...,...,...,...
151770,2180,797,1,2,0
151771,2180,799,1,2,1
151772,2185,797,2,None,0
151773,2185,799,1,2,0


In [24]:
data = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result/result_summary3.csv")
data_detailed = pd.read_csv(r"/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/result/result_detailed3.csv")
data_detailed

,time,player_id,path_from,path_to,block
0,5,1,1,2.0,28
1,5,3,1,2.0,29
2,5,4,1,2.0,29
3,5,2,2,1.0,28
4,5,5,2,1.0,29
...,...,...,...,...,...
151770,2180,797,1,2.0,0
151771,2180,799,1,2.0,1
151772,2185,797,2,NaN,0
151773,2185,799,1,2.0,0


In [25]:
# Run comprehensive network analysis
# This will analyze all lanes, generate plots, and save results

analysis = Analysis(data=data, detailed_data=data_detailed)

# Specify output directory (relative to notebook location)
output_directory = "/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output"

# Run full analysis
results = analysis.run_full_network_analysis(output_dir=output_directory)

# Access the results
print("\n📊 Summary of Results:")
print(f"Lane Metrics DataFrame shape: {results['lane_metrics'].shape}")
print(f"Network Stats Keys: {list(results['network_stats'].keys())}")
print(f"Output saved to: {results['output_dir']}")

📁 Output directory: /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output/analysis_20251223_101003
🔍 Found 2 lanes with vehicle data


Processing lanes:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_6432/1091613621.py:153: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  total_delay = np.trapz(queue_length, time_range)
/tmp/ipykernel_6432/1091613621.py:367: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1, 0].boxplot(box_data, labels=['Wait Time', 'Travel Time'], patch_artist=True)
/tmp/ipykernel_6432/1091613621.py:580: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans Mono.
  plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')
/tmp/ipykernel_6432/1091613621.py:580: UserWarning: Glyph 128663 (\N{AUTOMOBILE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "comprehensive_dashboard.png"), dpi=200, bbox_inches='tight')


📄 Saved CSV files to /mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/analysis_output/analysis_20251223_101003/summary

TRAFFIC SIMULATION ANALYSIS REPORT
Generated: 2025-12-23 10:10:10

NETWORK-WIDE STATISTICS
----------------------------------------
Total Trips Completed: 800
Simulation Duration: 2190 seconds
Throughput: 1315 vehicles/hour

WAIT TIME STATISTICS
----------------------------------------
Average Wait Time: 159.72 seconds
Median Wait Time: 159.50 seconds
Std Dev Wait Time: 92.70 seconds
Min Wait Time: 0.00 seconds
Max Wait Time: 328.00 seconds
25th Percentile: 80.00 seconds
75th Percentile: 239.00 seconds
95th Percentile: 304.00 seconds

TRAVEL TIME STATISTICS
----------------------------------------
Average Travel Time: 946.58 seconds
Median Travel Time: 946.00 seconds
Std Dev Travel Time: 463.52 seconds
Min Travel Time: 150.00 seconds
Max Travel Time: 1790.00 seconds
25th Percentile: 548.25 seconds
75th Percentile: 1344.00 seconds
95th Percentile:

In [8]:
print(hello)

NameError: name 'hello' is not defined